DeepBlink is a 2D CNN to detect spots:  

### Installation 
```
conda create -n starfish python=3.7  
conda activate starfish  
```
(if GPUs are available install cuda and tf2_gpu according to the nvidia drive)  
```
conda install -c bbquercus deepblink  
```

the network only works on 2d images
but they support 3d by offering a non maximal suppression:  
https://github.com/BBQuercus/deepBlink/blob/master/examples/3d_prediction.ipynb  
(used below)


In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
tf.config.experimental.list_physical_devices()

In [ ]:
is_gpu = len(tf.config.experimental.list_physical_devices('GPU')) > 0
is_gpu

In [ ]:
from glob import glob
import os
import pandas as pd
import tifffile as tif
import numpy as np
import time

In [ ]:
import deepblink as pink
from skimage import io
import trackpy as tp
import subprocess

#### Load pretrained model
We trained our own network, but results were not as good on the simulated data.  
and tested the pretrained netwrok "smfish.n5, but its not as good.  

In [ ]:
model = pink.io.load_model("particle.h5")

## Simulated Data
To analyze accuracy

#### Set Paths

In [ ]:
org_ims_dir = 'PATH/TO/IMAGES'
result_dir_simul = 'results_simul'

#### Load Images

In [ ]:
all_tifs = [f for f in glob(os.path.join(ss_dir, '*', '*spots*.tif')) if "3000spots" not in f]

In [ ]:
all_csvs = [f'{f[:-4]}.loc' for f in all_tifs]

In [ ]:
## if an image is missing a loc file
to_del = []
for i,f in enumerate(all_csvs):
    if not os.path.exists(f):
        to_del.append(i)

In [ ]:
for i in reversed(to_del):
    del all_csvs[i]
    del all_tifs[i]

save a list of corresponding filename and file number in the list.
will be used for all analysis

In [ ]:
## get imidiate dir and file name:
all_tifs = ["/".join(f.split('/')[-2:]) for f in all_tifs]
df_name_nums = pd.DataFrame({'name':all_tifs}) #'num': range(len(all_tifs)) ,
df_name_nums.to_csv('ims_and_corresponding_num.csv')

#### Predict

In [ ]:
os.makedirs(result_dir_simul)

In [ ]:
radius_refinement = 3
pad_width = radius_refinement

In [ ]:
times = []
dfs = []

for i,f in enumerate(all_tifs):
    im_3d = tif.imread(os.path.join(org_ims_dir, f))
    
    df = pd.DataFrame()
    
    time_im = 0

    for slice, image_curr in enumerate(im_3d):
        
        beg_time = time.time()
        
        # deepBlink prediction
        yx = pink.inference.predict(image=image_curr, model=model)
        y, x = yx.T.copy()
        
        time_im += (time.time() - beg_time)

        # pad to avoid error for spot close to the edges
        yx = yx + pad_width
        image_curr = np.pad(
            image_curr, pad_width=pad_width, mode='constant', constant_values = 0
        )

        # Refinement with trackpy
        df_curr = tp.refine_com(
            raw_image=image_curr, image=image_curr, radius=radius_refinement, coords=yx
        )
        df_curr["x"] = x
        df_curr["y"] = y
        df_curr["slice"] = slice
        df = df.append(df_curr, ignore_index=True)
        
    times.append(time_im)
        
    df.to_csv(os.path.join(result_dir_simul ,f'{i}.csv'), index=False)
    dfs.append(df)

In [ ]:
pd.DataFrame(times).to_csv(os.path.join(result_dir_simul ,f'times.csv'))

#### Unite spots (2D to 3D)

In [ ]:
search_range = 2
gap_frames = 0
min_frames = 2

In [ ]:
dfs_clean = []
times1 = []

for df in dfs:
    if not df.empty:

        beg_time = time.time()
        
        track = tp.link(df.rename({"slice": "frame"}, axis=1), 
                         search_range=search_range, memory=gap_frames)
        track = tp.filter_stubs(track, threshold=min_frames
                                ).rename({"frame": "slice"}, axis=1)

        # Index of brightest particles
        idx = track.groupby(["particle"])["mass"].transform(max) ==track["mass"]
        df_nms = track[idx]

        # Remove tracks from all spots
        df_without_track = df[
            ~df.set_index(["x", "y", "slice", "mass"]).index.isin(
                track.set_index(["x", "y", "slice", "mass"]).index
            )
        ]

        # Add back nms (brightest spots)
        df_clean = pd.concat([df_nms, df_without_track]).reset_index(drop=True)

        times1.append(time.time()-beg_time)
        
        dfs_clean.append(df_clean)
        #print(df_clean.shape)
    else:
        dfs_clean.append(0)
        times1.append(np.nan)


In [ ]:
pd.DataFrame(times1).to_csv(os.path.join(result_dir_simul ,f'times1.csv'))

#### Compare to GT

In [ ]:
## Leo's code:

# This function compares two arrays:
# Unmod = ground truth array
# More_than = detections from one of the programs

#Function checks if points in More_than are close/match points in GT array (under certain distance)

#Returns: 
# # of undetected ground truth points
# # spurious detections
# and average distance between detection and associated points

def profile_detections(unmod, more_than):

    min_dist = 2

    distance_arr = []

    removedItems = True
    euc_dist = 0

    while (removedItems and len(more_than) != 0 and len(unmod) != 0 ):
        #print("loop")

        minDist = 10000
        minIndexUnmod = -1
        minIndexMore_Than = -1
        counter = 0
        kd_copy = copy.deepcopy(more_than)
        kdtree = spatial.KDTree(kd_copy)

        for item in unmod:
            distance,index = kdtree.query(item) # a new KD tree is made
            if ( distance < minDist ):
                minDist = distance
                minIndexUnmod = counter
                minIndexMore_Than = index
                #print(minDist, counter, item)
            counter = counter + 1

        if ( minDist < min_dist): # if less than min dist
            more_than = np.delete(more_than, minIndexMore_Than, axis = 0 ) # delete mod ind
            unmod = np.delete(unmod,minIndexUnmod, axis = 0) #delete unmod ind
            #print(len(more_than),distance) # sanity checkd
            removedItems = True
            distance_arr.append(minDist) # if we want to extrat stat ig

        else:
            removedItems = False
    if (len(distance_arr) >0):
        euc_dist = np.mean(np.asarray(distance_arr))
        
    return(len(unmod), len(more_than), euc_dist)


In [ ]:
diff_results_list = []

for idx in df_name_nums.index:

    ## GT ##
    gt_path = df_name_nums.at[idx,'name']
    gt_dir = os.path.join('..', 'Selected_simulation')

    gt_path = os.path.join(gt_dir, gt_path[:-4] + '.loc')
        
    df = pd.read_csv(gt_path, sep = "\s+", header=None)
    gt_spots = df.to_numpy()[:,:-1] - np.array([0.5,0.5,1])

    df_db = dfs_clean[idx]
    
    if isinstance(df_db,int):
        continue
        
    detected_spots = df_db[df_db.particle.notna()][["y","x","slice"]].to_numpy()    

    diff_results = list(profile_detections(gt_spots, detected_spots))
    
    n_spots = int(gt_path.split("spots")[0].split("_")[-1])
    
    diff_results.insert(0, n_spots)
    diff_results = ",".join([str(d_r) for d_r in diff_results])
    
    diff_results_list.append(diff_results)

In [ ]:
with open(os.path.join(result_dir_simul, 'results_nspots_missed_overdetected_distance.txt'),'w') as f:
    f.write("\n".join(diff_results_list))